# *Python for Geoscience*

## 01 - Data Management

### A - Indexing & Binning Data

Author: Connor Doyle

Purpose: Indexing data according to pre-defined ranges.

Examples/Use Case: Assigning the names of formations to depths from a well.

Geological data is often associated with depth values - in many cases, this geological data can be assigned to stratigraphic hierarchies. For example, we may have a core analysis dataset from a range of subsurface depths, a set of drill-stem tests at various intervals to test for flow, or even the location of casing points associated with the well while it was drilled. However, once we have a set of a stratigraphic tops, whether identified from purely rock composition or from biostratigraphical analyses, we often need to place other data within the context of this stratigraphy. For example, we may have a core dataset that consists only of Depth, CPOR (Porosity), and KLH (Horizontal Klinkenberg Permeability) - but we need to know which formation they came from!

This code is therefore a simple way of assigning a 'dictionary' of top and base zones from our stratigraphic column, and subsequently applying that dictionary to our point depths associated with our core dataset. This can be particularly useful when plotting core analysis data within a scatter plot (see 02 - B - Scatter Plots in this Python for Geoscience series for more info on how to do this), as we can then colour the different points by their formation.

First, let's get started with importing the only library we need for this script - pandas!

## Import libraries

In [1]:
import pandas as pd

## Loading data to a dataframe 

Data are loaded here from either an Excel or csv file. The conditional statements below are written such that the pandas read function used will change depending on the filetype.

In [4]:
### Filepath Definition ###
depth_data = 'Data.xlsx'
range_data = 'Dictionary.xlsx'

### Filetype Handling ###

# If statement for dealing with multiple filetypes. 

# Range data
if range_data.endswith('csv'):
    print(f'{range_data} loaded from csv')
    range_df = pd.read_csv(f'{range_data}')
elif range_data.endswith('xlsx'):
    print(f'{range_data} loaded from Excel')
    range_df = pd.read_excel(f'{range_data}')
    
# Depth data
if depth_data.endswith('csv'):
    print(f'{depth_data} loaded from csv')
    depth_df = pd.read_csv(f'{depth_data}')
elif depth_data.endswith('xlsx'):
    print(f'{depth_data} loaded from Excel')
    depth_df = pd.read_excel(f'{depth_data}')

Dictionary.xlsx loaded from Excel
Data.xlsx loaded from Excel


## Display column names for use in input

Let's take a quick look at what our data looks like, and what columns we can use.

Our depth dataframe contains two columns - one for a site, and one for the depth of that site. But we want to know what zone it's related to - so let's take a look at that too!

In [5]:
depth_df.head()

,Site,Depth
0,A,1
1,A,3
2,A,5
3,A,61
4,A,23


This is our 'dictionary' of zones - their top and base for each site. So, we want to use this to define where our depths are from the previous dataframe.

In [6]:
range_df.head()

,Site,Zone,Top,Base
0,A,A1,1,5
1,A,A2,5,10
2,A,A3,10,20
3,B,B1,50,100
4,B,B2,100,200


There are also other ways of seeing what headers we have available to use - this example simply prints all of our headers out for both dataframes.

In [7]:
# Print statement to allow user to see column names and match nomenclature
print('Depth Columns')
for col in depth_df.columns:
    print(col)
print('Range Columns')
for col in range_df.columns:
    print(col)

Depth Columns
Site
Depth
Range Columns
Site
Zone
Top
Base


## Index Data

Let's begin with assigning our depths a zone for each site. First, we can define a function to carry this out.

In [8]:
# Function to assign zones based on a top and base depth
def assign_zone(depth, site, range_df):
    
    site_zone_depths = range_df[range_df['Site'] == site] # Filter only on rows for each site in ranges_df
    
    for _, row in site_zone_depths.iterrows(): # For each row in site_zone_depths
        if row['Top'] <= depth <= row['Base']:
            return row['Zone'] # If found, return the zone

Now we have a function to carry out our zone assignment to each depth, we need some way of storing this new data, and we need to carry out the function on our depth data.

In [9]:
zones = [] # Empty list to store zones for later output

for _, row in depth_df.iterrows(): # For all rows in our depth dataframe
    zone = assign_zone(row['Depth'], row['Site'], range_df)
    zones.append(zone)

depth_df['Zone'] = zones # Create new column in depth dataframe

depth_df.to_excel(r'Assigned_Zones.xlsx', index=False) # Output to excel

Peeking at our new depth dataframe subsequently gives us the following:

In [11]:
depth_df.head() # Optional print step to check outputs

,Site,Depth,Zone
0,A,1,A1
1,A,3,A1
2,A,5,A1
3,A,61,None
4,A,23,None
